# Interpolation of f1 CAFE hybrid levels to isobaric levels
### NOTE: must run `ulimit -s unlimited` prior to running this notebook

In [1]:
import glob
import subprocess
import itertools
import xarray as xr
import numpy as np
import time

## Location of cafe grid info

In [2]:
cafe_grid_info = '/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/shared/cafe_grid_info.nc3.nc '

## Get list of files to convert

In [3]:
years = np.arange(2002,2017)
months = np.arange(1,13)
ensembles = np.arange(1,12)
folder = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/'
files = 'atmos_daily*01.nc'

filelist = []
for year, month, ensemble in itertools.product(years, months, ensembles):
    filelist.append(sorted(glob.glob(folder + 'yr' + str(year) 
                                            + '/mn' + str(month) 
                                            + '/OUTPUT.' + str(ensemble) 
                                            + '/' + files)))

filelist = list(itertools.chain.from_iterable(filelist))

## Levels to interpolate to

In [4]:
levels = [20000, 30000, 40000, 45000, 50000, 55000, 60000, 70000, 85000, 100000]

## Loop over each file and interpolate

In [ ]:
for file in filelist:
    
    # Add ak and bk fields to atmospheric output files -----
    command = '/apps/nco/4.4.4/bin/ncks -A -v ak,bk,zsurf ' + cafe_grid_info + file
    out = subprocess.call(command, shell=True)
    if (out != 0) & (out != 1):
        print('Failed to add alphas and betas:')
        print(file, '\n')
        
    time.sleep(1.0) 
    
    # Rename ak with pk as required by PLEV executable -----
    command = '/apps/nco/4.4.4/bin/ncrename -v ak,pk ' + file
    out = subprocess.call(command, shell=True)
    if (out != 0) & (out != 1):
        print('Failed to rename alpha:')
        print(file, '\n')
        
    # Delete old plevel file if it exists -----
    command = 'rm -r ' + file[:file.index('.nc')] + '.plevel' + file[file.index('.nc'):]
    out = subprocess.call(command, shell=True)
        
    time.sleep(1.0) 
    
    # Perform the interpolation -----
    command = './plevel.sh -a -i ' + file + \
              ' -o ' + file[:file.index('.nc')] + '.tmp' + file[file.index('.nc'):] + \
              ' -p ' + '"' + ' '.join(map(str, levels)) + '"' + \
              ' hght'
    out = subprocess.call(command, shell=True)
    if (out != 0) & (out != 1):
        print(f'Regridding failed, error {out}:')
        print(file, '\n')
    
    time.sleep(1.0) 
    
    # Convert to netcdf-4 -----
    command = 'ncks -4 -L 1 ' + file[:file.index('.nc')] + '.tmp' + file[file.index('.nc'):] + ' ' \
                              + file[:file.index('.nc')] + '.plevel' + file[file.index('.nc'):]
    out = subprocess.call(command, shell=True)
    if (out != 0) & (out != 1):
        print('Failed to convert to netcdf-4:')
        print(file, '\n')
    
    time.sleep(1.0) 
        
    # Delete temporary file -----
    command = 'rm -r ' + file[:file.index('.nc')] + '.tmp' + file[file.index('.nc'):]
    out = subprocess.call(command, shell=True)